In [1]:
%matplotlib inline
%config IPCompleter.greedy=True

import copy
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import math as math
import scipy.stats as st
import auxiliary_functions as aux
from scipy.optimize import curve_fit


pd.options.display.max_rows = 10

# Run the script to load the data (the series and their segmentation) and to delete all segments with indeterminate stage or whose max frequency is below 85%
from load_data import load_data, redo_classification
data, seg_res = load_data()
# Reclassificates the segments using the 4 stage system.
seg_res = redo_classification(seg_res, data)
# Load the increments data
from load_increments import load
increments = load(data, seg_res, 4)


# Aglutinates the segments statistics of all the subjects, distinguishing them only by their groups.
from aglutinates_statistics import aglutinates
means, medians, variances, lengths, t_i, t_f, delta_t = aglutinates(data, seg_res, 4)

In [2]:
groups_titles = ["Hypertensive", "Normotensive", "Proband"]
series_titles = ["SBP", "DBP", "BBI-P", "BBI-EKG"]
measures_titles = ["Mean", "Variance", "Length", r'T_i', r'T_f', r'\Delta_t', "Median", "Increments"]
stages = ["Vigil", "Light", "Deep", "REM"]
measures = [means, variances, lengths, t_i, t_f, delta_t, medians, increments]

#### Boxplot

In [3]:
# Boxplot

path = "Plots/Measures/4 Groups/Boxplot/"

# Creates all the figures, one for each parameter
fig_mean, axs_mean = plt.subplots(nrows = 4, ncols = 3)
fig_mean.set_size_inches(20, 20)
fig_mean.subplots_adjust(hspace = 0.3, wspace = 0.3)

fig_var, axs_var = plt.subplots(nrows = 4, ncols = 3)
fig_var.set_size_inches(20, 20)
fig_var.subplots_adjust(hspace = 0.3, wspace = 0.3)

fig_lg, axs_lg = plt.subplots(nrows = 4, ncols = 3)
fig_lg.set_size_inches(20, 20)
fig_lg.subplots_adjust(hspace = 0.3, wspace = 0.3)

fig_ti, axs_ti = plt.subplots(nrows = 4, ncols = 3)
fig_ti.set_size_inches(20, 20)
fig_ti.subplots_adjust(hspace = 0.3, wspace = 0.3)

fig_tf, axs_tf = plt.subplots(nrows = 4, ncols = 3)
fig_tf.set_size_inches(20, 20)
fig_tf.subplots_adjust(hspace = 0.3, wspace = 0.3)

fig_dt, axs_dt = plt.subplots(nrows = 4, ncols = 3)
fig_dt.set_size_inches(20, 20)
fig_dt.subplots_adjust(hspace = 0.3, wspace = 0.3)

fig_med, axs_med = plt.subplots(nrows = 4, ncols = 3)
fig_med.set_size_inches(20, 20)
fig_med.subplots_adjust(hspace = 0.3, wspace = 0.3)

fig_saltos, axs_saltos = plt.subplots(nrows = 4, ncols = 3)
fig_saltos.set_size_inches(20, 20)
fig_saltos.subplots_adjust(hspace = 0.3, wspace = 0.2)

# Creates all the lists for iteration
ylabels = ["log(Mean)", "log(Var)", "log(L)", r'T_i', r'T_f', r'\Delta_t', "log(Median)", "abs(Increments)"]
colors = ["whitesmoke", "lightgrey", "gray", "dimgray"]
figs = [fig_mean, fig_var, fig_lg, fig_ti, fig_tf, fig_dt, fig_med, fig_saltos]
axs  = [axs_mean, axs_var, axs_lg, axs_ti, axs_tf, axs_dt, axs_med, axs_saltos]

# Boxplot
for g in range(3):
    for s in range(4):
        for i in range(8):
            bplot = axs[i][s][g].boxplot(measures[i][g][s], labels = stages, patch_artist = True, showfliers = False, showmeans = True)
            axs[i][s][g].set_ylabel(ylabels[i], fontsize = 12)
            axs[i][s][g].set_xlabel("Stages", fontsize = 12)
            axs[i][s][g].set_title(groups_titles[g]+" - "+series_titles[s], fontsize = 14)    
            plt.setp(bplot['medians'], color = 'black')
            plt.setp(bplot['means'], color = 'r')
            for patch, color in zip(bplot['boxes'], colors):
                patch.set(facecolor = color)
                
                     
for i in range(8):
    figs[i].savefig(path+measures_titles[i]+".png")
    plt.close(figs[i])

#### Empirical CDF

In [5]:
# Empirical CDF

path = "Plots/Measures/4 Groups/EmpiricalCDF/"

# Create figures
fig_mean_cdf, axs_mean_cdf = plt.subplots(nrows = 4, ncols = 3)

fig_var_cdf, axs_var_cdf   = plt.subplots(nrows = 4, ncols = 3)

fig_lg_cdf, axs_lg_cdf     = plt.subplots(nrows = 4, ncols = 3)

fig_ti_cdf, axs_ti_cdf     = plt.subplots(nrows = 4, ncols = 3)

fig_tf_cdf, axs_tf_cdf     = plt.subplots(nrows = 4, ncols = 3)

fig_dt_cdf, axs_dt_cdf     = plt.subplots(nrows = 4, ncols = 3)

fig_med_cdf, axs_med_cdf   = plt.subplots(nrows = 4, ncols = 3)

fig_inc_cdf, axs_inc_cdf   = plt.subplots(nrows = 4, ncols = 3)

# Creates all the lists for iteration
colors    = ["gold", "steelblue", "purple", "black"]
ylabels = ["Fn(log(Mean))", "Fn(log(Var))", "Fn(log(L))", r'T_i', r'T_f', r'\Delta_t', "Fn(log(Median))", "Fn(abs(Increment))"]
axs = [axs_mean_cdf, axs_var_cdf, axs_lg_cdf, axs_ti_cdf, axs_tf_cdf, axs_dt_cdf, axs_med_cdf, axs_inc_cdf]
figs = [fig_mean_cdf, fig_var_cdf, fig_lg_cdf, fig_ti_cdf, fig_tf_cdf, fig_dt_cdf, fig_med_cdf, fig_inc_cdf]

# Configure figure sizes and axis distances
for item in figs:
    item.set_size_inches(20, 20)
    item.subplots_adjust(hspace = 0.3, wspace = 0.2)

# Configure title sizes
for ax_arr in axs:
    for i in range(4):
        for j in range(3):
            ax_arr[i][j].title.set_fontsize(14)
            ax_arr[i][j].xaxis.label.set_fontsize(12)
            ax_arr[i][j].yaxis.label.set_fontsize(12)
            
# Plot all curves
for m in range(8):
    for g in range(3):
        for s in range(4):
            measure = measures[m][g][s]
            
            # Computes the cumulative density function
            Fn = [[], [], [], [], [], []]
            for i in range(4):
                measure[i].sort()
                Fn[i] = -np.sort(-np.arange(len(measure[i]))/float(len(measure[i])))
             
            for i in range(4):
                ax = axs[m][s][g]
                ax.scatter(measure[i], Fn[i], color = colors[i], s = 5, label = stages[i], marker = '.')
                ax.set_xlabel(measures_titles[m])
                ax.set_ylabel('1 - '+ylabels[m])
                ax.set_title(groups_titles[g]+": CDF - "+measures_titles[m]+" - "+series_titles[s])
                ax.legend(fontsize = 10, markerscale = 7)
            
    figs[m].savefig(path+measures_titles[m]+".png")
    plt.close(figs[m])

#### Probability Density Function

In [6]:
# Density Function

path = "Plots/Measures/4 Groups/Density/"

# Create figures
fig_mean, axs_mean = plt.subplots(nrows = 4, ncols = 3)

fig_var, axs_var = plt.subplots(nrows = 4, ncols = 3)

fig_lg, axs_lg = plt.subplots(nrows = 4, ncols = 3)

fig_ti, axs_ti = plt.subplots(nrows = 4, ncols = 3)

fig_tf, axs_tf = plt.subplots(nrows = 4, ncols = 3)

fig_dt, axs_dt = plt.subplots(nrows = 4, ncols = 3)

fig_med, axs_med = plt.subplots(nrows = 4, ncols = 3)

fig_inc, axs_inc = plt.subplots(nrows = 4, ncols = 3)


# Creates all the lists for iteration
colors     = ["gold", "steelblue", "purple", "black"]
ylabels = ["p(log(Mean))", "p(log(Var))", "p(log(L))", "T_i", "T_f", "D_t", "p(log(Median))", "p(abs(Increment))"]
axs  = [axs_mean, axs_var, axs_lg, axs_ti, axs_tf, axs_dt, axs_med, axs_inc]
figs = [fig_mean, fig_var, fig_lg, fig_ti, fig_tf, fig_dt, fig_med, fig_inc]

# Configure figure sizes and axis distances
for item in figs:
    item.set_size_inches(20, 20)
    item.subplots_adjust(hspace = 0.3, wspace = 0.5)

# Configure title sizes
for ax_arr in axs:
    for i in range(4):
        for j in range(3):
            ax_arr[i][j].title.set_fontsize(3)
            ax_arr[i][j].xaxis.label.set_fontsize(6)
            ax_arr[i][j].yaxis.label.set_fontsize(6)
            

# Plot all curves
for m in range(8):
    for g in range(3):
        for s in range(4):
            measure = measures[m][g][s]
            
            for i in range(4):
                axis = axs[m][s][g]
                sns.distplot(measure[i], hist = False, kde = True, kde_kws = {'linewidth' : 3}, ax = axis,
                             color = colors[i], label = stages[i])
                axis.set_xlabel(measures_titles[m])
                axis.set_ylabel(ylabels[m])
                axis.set_title("p(x) - "+measures_titles[m]+" - "+groups_titles[g]+" - "+series_titles[s])
            
    figs[m].savefig(path+measures_titles[m]+".png")
    plt.close(figs[m])